<a href="https://colab.research.google.com/github/Redcxx/ucl-master-project/blob/master/pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Settings

In [2]:
#@title pip installs

%pip install pydrive2 > /dev/null
%pip install torchinfo > /dev/null

In [3]:
#@title imports

import sys
import os
import random
import functools
from datetime import datetime
from pathlib import Path
from pprint import pprint

import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from torchinfo import summary
from PIL import Image


In [3]:
#@title constants

RUN_ID = datetime.now().strftime('%Y-%m-%d-%A-%Hh-%Mm-%Ss')
RANDOM_SEED = 42
WORKING_FOLDER_NAME = 'MasterProject'  # will be created on google drive
PYDRIVE2_SETTING_FILE = 'settings.yaml'


print(f'RUN_ID: {RUN_ID}')
print(f'RANDOM_SEED: {RANDOM_SEED}')
print(f'WORKING_FOLDER_NAME: {WORKING_FOLDER_NAME}')

RUN_ID: 2022-05-27-Friday-06h-30m-52s
RANDOM_SEED: 42
WORKING_FOLDER_NAME: MasterProject


In [4]:
#@title colab & jupyter notebook settings for saving file

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import files, drive

    drive_dir = '/content/drive'
    drive.mount(drive_dir)

    working_dir = os.path.join(drive_dir, 'My Drive', WORKING_FOLDER_NAME)
    Path(working_dir).mkdir(parents=True, exist_ok=True)  # create directory if not exists on google drive

    def save_file(file_name):
        files.download(file_name)  # save locally

        # save on google drive
        with open(file_name, 'rb') as src_file:
            with open(os.path.join(working_dir, file_name), 'wb') as dest_file:
                dest_file.write(src_file.read())

else:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive

    def ensure_folder_on_drive(drive, folder_name):
        folders = drive.ListFile({
            # see https://developers.google.com/drive/api/guides/search-files
            'q': "mimeType = 'application/vnd.google-apps.folder'"
        }).GetList()

        folders = filter(lambda folder: folder['title'] == folder_name, folders)

        if len(folders) == 1:
            return folders[0]
        
        if len(folders) > 1:
            pprint(folders)
            raise AssertionError('Multiple Folders of the same name detected')

        # folder not found, create a new one at root
        print(f'Folder: {folder_name} not found, creating at root')

        folder = drive.CreateFile({
            'title': folder_name, 
            # "parents": [{
            #     "kind": "drive#fileLink", 
            #     "id": parent_folder_id
            # }],
            "mimeType": "application/vnd.google-apps.folder"
        })
        folder.Upload()
        return folder


    g_auth = GoogleAuth(settings_file=PYDRIVE2_SETTING_FILE, http_timeout=None)
    g_auth.LocalWebserverAuth(host_name="localhost", port_numbers=None, launch_browser=True)
    drive = GoogleDrive(g_auth)

    folder = ensure_folder_on_drive(drive, WORKING_FOLDER_NAME)    

    def save_file(file_name):
        file = drive.CreateFile({
            'title': file_name,
            'parents': [{
                'id': folder['id']
            }]
        })
        file.SetContentFile(file_name)
        file.Upload()  # save to google drive
        file.GetContentFile(file_name)  # save locally    

Mounted at /content/drive


In [5]:
#@title misc

# reproducibility
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

# training device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [6]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# Models

In [43]:
#@title Unet Block

class UnetBlock(nn.Module):
    def __init__(
        self, 
        in_filters, out_filters,

        submodule=None, 
        sub_in_filters=None, 
        sub_out_filters=None, 
        sub_skip_connection=False, 

        skip_connection=True, 
        dropout=nn.Dropout, 
        in_norm=nn.BatchNorm2d, out_norm=nn.BatchNorm2d, 
        in_act=nn.LeakyReLU, out_act=nn.ReLU,
    ):
        super().__init__()

        if submodule is None:
            sub_in_filters = in_filters
            sub_out_filters = in_filters
            sub_skip_connection = False
        
        conv_common_args = {
            'kernel_size': 4, 
            'stride': 2, 
            'padding': 1,
            'bias': in_norm.func != nn.BatchNorm2d if type(in_norm) == functools.partial else in_norm != nn.BatchNorm2d  # batch norm has bias
        }

        layers = []

        # encoder
        layers.append(nn.Conv2d(in_channels=in_filters, out_channels=sub_in_filters, **conv_common_args))

        if in_norm:
            layers.append(in_norm(sub_in_filters))

        if in_act:
            layers.append(in_act())

        
        # submodule
        if submodule:
            layers.append(submodule)


        # decoder
        if sub_skip_connection:
            layers.append(nn.ConvTranspose2d(in_channels=sub_out_filters * 2, out_channels=out_filters, **conv_common_args))
        else:
            layers.append(nn.ConvTranspose2d(in_channels=sub_out_filters    , out_channels=out_filters, **conv_common_args))

        if out_norm:
            layers.append(out_norm(out_filters))
        
        if dropout:
            layers.append(dropout())
        
        if out_act:
            layers.append(out_act())
        
        self.model = nn.Sequential(*layers)

        self.skip_connection = skip_connection
    
    def forward(self, x):
        if self.skip_connection:
            return torch.cat([x, self.model(x)], dim=1)
        else:
            return self.model(x)

In [44]:
summary(
    UnetBlock(in_filters=64, out_filters=64, submodule=None), 
    input_size=(16, 64, 16, 16),
    col_names=['output_size', 'num_params', 'mult_adds']
)

Layer (type:depth-idx)                   Output Shape              Param #                   Mult-Adds
UnetBlock                                --                        --                        --
├─Sequential: 1-1                        [16, 64, 16, 16]          --                        --
│    └─Conv2d: 2-1                       [16, 64, 8, 8]            65,536                    67,108,864
│    └─BatchNorm2d: 2-2                  [16, 64, 8, 8]            128                       2,048
│    └─LeakyReLU: 2-3                    [16, 64, 8, 8]            --                        --
│    └─ConvTranspose2d: 2-4              [16, 64, 16, 16]          65,536                    268,435,456
│    └─BatchNorm2d: 2-5                  [16, 64, 16, 16]          128                       2,048
│    └─Dropout: 2-6                      [16, 64, 16, 16]          --                        --
│    └─ReLU: 2-7                         [16, 64, 16, 16]          --                        --
Total para

In [47]:
#@title Generator

class Generator(nn.Module):

    def __init__(self, config):
        super().__init__()

        # dependency injection
        batch_norm = functools.partial(nn.BatchNorm2d, affine=True, track_running_stats=True)
        relu = functools.partial(nn.ReLU, inplace=True)
        leaky_relu = functools.partial(nn.LeakyReLU, inplace=True, negative_slope=0.2)
        dropout = functools.partial(nn.Dropout, p=0.5)
        tahn = nn.Tanh
        
        # build model recursively inside-out
        blocks = config['blocks'][::-1]  

        self.model = None

        # build innermost block
        self.model = UnetBlock(
            in_filters=blocks[0]['filters'], 
            out_filters=blocks[0]['filters'],

            submodule=None, 
            sub_in_filters=None, 
            sub_out_filters=None,
            sub_skip_connection=False,

            skip_connection=False,
            dropout=dropout if blocks[0]['dropout'] else None,
            in_norm=None, out_norm=False,
            in_act=relu, out_act=relu
        )
        
        # build between blocks
        for i, layer in enumerate(blocks[1:], 1):
            self.model = UnetBlock(
                in_filters=layer['filters'], 
                out_filters=layer['filters'],

                submodule=self.model, 
                sub_in_filters=blocks[i-1]['filters'], 
                sub_out_filters=blocks[i-1]['filters'],
                sub_skip_connection=blocks[i-1]['skip_connection'],

                skip_connection=blocks[i]['skip_connection'],
                dropout=dropout if layer['dropout'] else None,
                in_norm=batch_norm, out_norm=batch_norm,
                in_act=leaky_relu, out_act=relu
            )
        
        # build outermost block
        self.model = UnetBlock(
            in_filters=config['in_channels'],
            out_filters=config['out_channels'],

            submodule=self.model,
            sub_in_filters=blocks[-1]['filters'], 
            sub_out_filters=blocks[-1]['filters'],
            sub_skip_connection=blocks[-1]['skip_connection'],

            skip_connection=False, 
            dropout=None,
            in_norm=None, out_norm=None,
            in_act=leaky_relu, out_act=tahn
        )

    def forward(self, x):
        return self.model(x)

In [48]:
#@title Generator summary

generator_config = {
    'in_channels': 3,
    'out_channels': 3,
    'blocks': [
    {
        'filters': 64,
        'dropout': False,
        'skip_connection': True
    }, 
    {
        'filters': 128,
        'dropout': False,
        'skip_connection': True
    }, 
    {
        'filters': 256,
        'dropout': False,
        'skip_connection': True
    }, 
    {
        'filters': 512,
        'dropout': False,
        'skip_connection': True
    }, 
    {
        'filters': 512,
        'dropout': True,
        'skip_connection': True
    }, 
    {
        'filters': 512,
        'dropout': True,
        'skip_connection': True
    }, 
    {
        'filters': 512,
        'dropout': True,
        'skip_connection': False
    }]
}

summary(
    Generator(generator_config),
    input_size=(16, 3, 256, 256),
    col_names=['output_size', 'num_params', 'mult_adds'],
    depth=24
)

Layer (type:depth-idx)                                                                                         Output Shape              Param #                   Mult-Adds
Generator                                                                                                      --                        --                        --
├─UnetBlock: 1-1                                                                                               [16, 3, 256, 256]         --                        --
│    └─Sequential: 2-1                                                                                         [16, 3, 256, 256]         --                        --
│    │    └─Conv2d: 3-1                                                                                        [16, 64, 128, 128]        3,072                     805,306,368
│    │    └─LeakyReLU: 3-2                                                                                     [16, 64, 128, 128]        --               

In [49]:
class Discriminator(nn.Module):
    def __init__(self, config):
        super().__init__()

        # we do not use bias in conv2d layer if batch norm is used, because batch norm already has bias
        batch_norm = functools.partial(nn.BatchNorm2d, affine=True, track_running_stats=True)
        leaky_relu = functools.partial(nn.LeakyReLU, negative_slope=0.2, inplace=True)

        conv_common_args = {
            'kernel_size': 4, 
            'padding': 1,
        }

        blocks = config['blocks']
        layers = []

        # build first block
        layers += [
            nn.Conv2d(config['in_channels'], blocks[0]['filters'], stride=2, **conv_common_args),
            leaky_relu()
        ]

        # build between block
        prev_filters = blocks[0]['filters']
        for i, layer in enumerate(blocks[1:-1], 1):
            curr_filters = min(blocks[i]['filters'], blocks[0]['filters']*8)
            layers += [
                nn.Conv2d(prev_filters, curr_filters, bias=False, stride=2, **conv_common_args),
                batch_norm(curr_filters),
                leaky_relu()
            ]
            prev_filters = curr_filters

        # build last block
        curr_filters = min(blocks[-1]['filters'], blocks[0]['filters'] * 8)
        layers += [
            # stride = 1 for last block
            nn.Conv2d(prev_filters, curr_filters, stride=1, bias=False, **conv_common_args),
            batch_norm(curr_filters),
            leaky_relu(),
            # convert to 1 dimensional output
            nn.Conv2d(curr_filters, 1, stride=1, **conv_common_args)
        ]

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [50]:
discriminator_config = {
    'in_channels': 3,
    'out_channels': 3,
    'blocks': [
    {
        'filters': 64,
    }, 
    {
        'filters': 128,
    }, 
    {
        'filters': 256,
    }, 
    {
        'filters': 512,
    }]
}

summary(
    Discriminator(discriminator_config),
    input_size=(16, 3, 256, 256),
    col_names=['output_size', 'num_params', 'mult_adds'],
    depth=24
)

Layer (type:depth-idx)                   Output Shape              Param #                   Mult-Adds
Discriminator                            --                        --                        --
├─Sequential: 1-1                        [16, 1, 30, 30]           --                        --
│    └─Conv2d: 2-1                       [16, 64, 128, 128]        3,136                     822,083,584
│    └─LeakyReLU: 2-2                    [16, 64, 128, 128]        --                        --
│    └─Conv2d: 2-3                       [16, 128, 64, 64]         131,072                   8,589,934,592
│    └─BatchNorm2d: 2-4                  [16, 128, 64, 64]         256                       4,096
│    └─LeakyReLU: 2-5                    [16, 128, 64, 64]         --                        --
│    └─Conv2d: 2-6                       [16, 256, 32, 32]         524,288                   8,589,934,592
│    └─BatchNorm2d: 2-7                  [16, 256, 32, 32]         512                       8,

# Data

In [ ]:
%%bash

FILE="facades"

if [[ $FILE != "cityscapes" && $FILE != "night2day" && $FILE != "edges2handbags" && $FILE != "edges2shoes" && $FILE != "facades" && $FILE != "maps" ]]; then
  echo "Available datasets are cityscapes, night2day, edges2handbags, edges2shoes, facades, maps"
  exit 1
fi

if [[ $FILE == "cityscapes" ]]; then
    echo "Due to license issue, we cannot provide the Cityscapes dataset from our repository. Please download the Cityscapes dataset from https://cityscapes-dataset.com, and use the script ./datasets/prepare_cityscapes_dataset.py."
    echo "You need to download gtFine_trainvaltest.zip and leftImg8bit_trainvaltest.zip. For further instruction, please read ./datasets/prepare_cityscapes_dataset.py"
    exit 1
fi

echo "Specified [$FILE]"

URL=http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/$FILE.tar.gz
TAR_FILE=./datasets/$FILE.tar.gz
TARGET_DIR=./datasets/$FILE/
wget -N $URL -O $TAR_FILE > /dev/null
mkdir -p $TARGET_DIR
tar -zxvf $TAR_FILE -C ./datasets/ > /dev/null
rm $TAR_FILE

In [9]:
#@title util functions

IMG_EXTENSIONS = [
    '.jpg', '.jpeg',
    '.png', '.ppm', '.bmp',
    '.tif', '.tiff',
]

def is_image_file(filename):
    return any(filename.lower().endswith(ext) for ext in IMG_EXTENSIONS)

def get_all_image_paths(root):
    paths = []
    assert os.path.isdir(root)

    for root, _folders, filenames in sorted(os.walk(root)):
        for filename in filenames:
            if is_image_file(filename):
                paths.append(os.path.join(root, filename))
    
    return paths

In [7]:
class MyDataset(Dataset):

    def __init__(self, root):
        self.paths = sorted(get_all_image_paths(root))
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, i):
        A, B = self._split_input_output(self._read_im(self.paths[i]))

        transform = self._generate_transform()
        
        return transform(A), transform(B)

    def _read_im(self, path):
        return Image.open(path).convert('RGB')

    def _split_input_output(AB):
        w, h = AB.size
        w2 = int(w / 2)
        A = AB.crop((0, 0, w2, h))
        B = AB.crop((w2, 0, w, h))

        return A, B

    def _generate_transform(self):
        new_size = 286
        old_size = 256

        rand_x = random.randint(0, new_size - old_size)
        rand_y = random.randint(0, new_size - old_size)
        flip = random.random() > 0.5
        
        return transforms.Compose([
            transforms.Resize((new_size, new_size), interpolation=InterpolationMode.BICUBIC, antialias=True),
            transforms.Lambda(lambda im: self._crop(im, (rand_x, rand_y), (old_size, old_size))),
            transforms.Lambda(lambda im: self._flip(im, flip)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])


    def _flip(self, im, flip):
        if flip:
            return im.transpose(Image.FLIP_LEFT_RIGHT)
        return im

    def _crop(self, im, pos, size):
        return im.crop((pos[0], pos[1], pos[0] + size[0], pos[1] + size[1]))

In [10]:
batch_size = 1
shuffle = False
num_workers = 4

dataset = MyDataset('./datasets/facades/train')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)